# 데이터 분석 Agent

## Pandas agent

In [ ]:
!pip install -qU langchain-teddynote
!pip install -q langchain langchain-community langchain-openai
!pip install -q chromadb sentence-transformers
!pip install -q PyPDF2 pymupdf
!pip install -q gradio
!pip install -q tiktoken
!pip install -q openai langchain_experimental

In [8]:
!pip install -q langchain_experimental

In [ ]:
# 나눔고딕 폰트 설치 및 설정
!apt-get update -qq
!apt-get install fonts-nanum -qq
!fc-cache -fv
!rm ~/.cache/matplotlib -rf


import os

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 폰트 설정
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
fontprop = fm.FontProperties(fname=font_path, size=10)
plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False

In [21]:
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

def set_font(): 
    # 폰트 설정
    font_path='/System/Library/Fonts/AppleGothic.ttf'
    fontprop = fm.FontProperties(fname=font_path, size=10)
    plt.rcParams['font.family'] = 'AppleGothic'  # macOS
    plt.rcParams['axes.unicode_minus'] = False
    
set_font()

In [2]:
import os
# OpenAI API 키 설정 (사용자가 입력해야 함)
# from google.colab import userdata
# api_key=userdata.get('api_key')
# os.environ["OPENAI_API_KEY"] = api_key
# api_key2=userdata.get('api_key2')
# os.environ["LANGCHAIN_API_KEY"] = api_key2

from dotenv import load_dotenv

load_dotenv()
# OpenAI API 클라이언트 생성
OPENAPI_KEY = os.getenv("OPENAI_API_KEY")
LangSmith_KEY = os.getenv("LANGCHAIN_API_KEY")

# 2) LangSmith 연동 필수 환경변수
os.environ["LANGCHAIN_TRACING_V2"] = "true"      # 트레이싱 활성화
os.environ["LANGSMITH_ENDPOINT"]   = "https://api.smith.langchain.com"  # 기본값
os.environ["LANGSMITH_PROJECT"]    = "Agent_da_titanic"                 # 수업용 프로젝트명

In [6]:
import pandas as pd
df = pd.read_csv("./titanic_kor.csv", encoding='cp949')  # CSV 파일을 읽습니다.
# df = pd.read_excel("./data/titanic.xlsx") # 엑셀 파일도 읽을 수 있습니다.
df.head()

,생존,등급,성별,나이,형제자매,부모자녀,요금,탑승지코드,객실등급,성인여부,성인남성,선실코드,탑승지명,생존여부,개인탑승자
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [10]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.agents.agent_types import AgentType
from langchain_openai import ChatOpenAI
from langchain_teddynote.messages import AgentStreamParser
import seaborn as sns

sns.set_style("white")

agent = create_pandas_dataframe_agent(
    ChatOpenAI(model="gpt-4o-mini", temperature=0),
    df,
    verbose=False,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    allow_dangerous_code=True,
)

stream_parser = AgentStreamParser()


In [33]:
def ask(query):
    # 질의에 대한 답변을 출력합니다.
    # response = agent.stream({"input": query})

    # for step in response:
    #     stream_parser.process_agent_steps(step)
    response = agent.run(query)
    return response


In [32]:
ask("몇 개의 행이 있어?")


In [34]:
print(ask("남자와 여자의 생존율의 차이는 몇이야?"))


남자와 여자의 생존율의 차이는 약 -0.5531입니다. 이는 남자의 생존율이 여자의 생존율보다 약 55.31% 낮다는 것을 의미합니다.


In [ ]:
ask("남자 승객과 여자 승객의 생존율을 구한뒤 barplot 차트로 시각화 해줘")


In [ ]:
ask("1,2등급에 탑승한 10세 이하 어린 아이의 성별별 생존율을 구하고 시각화 하세요")


In [ ]:
ask("1,2등급에 탑승한 10세 이하 어린 아이의 생존율을 예측하고 결과 알려줘")

# UI with Gradio

In [ ]:
import gradio as gr
import io
from PIL import Image

class TitanicAnalyzer:
    def __init__(self, df):
        self.df = df
        self.agent = None
        if self.df is None:
            self.load_data()
        self.setup_agent()
    
    def load_data(self):
        
        """타이타닉 데이터 로드"""
        try:
            # 로컬 파일이 있으면 로드, 없으면 seaborn에서 로드
            if os.path.exists('./titanic_kor.csv'):
                self.df = pd.read_csv('./titanic_kor.csv')
            else:
                # seaborn의 타이타닉 데이터 로드
                self.df = sns.load_dataset('titanic')
            
            print(f"데이터 로드 완료: {self.df.shape}")
            print(f"컬럼: {list(self.df.columns)}")
            
        except Exception as e:
            print(f"데이터 로드 실패: {e}")
            # 기본 타이타닉 데이터 생성
            self.create_sample_data()
    
    
    def setup_agent(self):
        """pandas dataframe agent 설정"""
        try:
            # OpenAI API 키 설정 (환경변수에서 읽어옴)
            api_key = os.getenv('OPENAI_API_KEY')
            if not api_key:
                print("OpenAI API 키가 설정되지 않았습니다.")
                return

            # ChatOpenAI 모델 초기화
            llm = ChatOpenAI(
                temperature=0,
                model="gpt-4o-mini",
                openai_api_key=api_key
            )
            
            # pandas dataframe agent 생성
            self.agent = create_pandas_dataframe_agent(
                llm,
                self.df,
                verbose=True,
                agent_type=AgentType.OPENAI_FUNCTIONS,
                allow_dangerous_code=True,  # 코드 실행 허용
                handle_parsing_errors=True
            )
            
            print("Agent 설정 완료")
            
        except Exception as e:
            print(f"Agent 설정 실패: {e}")
    
    def analyze_query(self, query):
        """사용자 질의 분석"""
        if not self.agent:
            return "AI 에이전트가 설정되지 않았습니다. OpenAI API 키를 확인해주세요."
        
        if not query.strip():
            return "질문을 입력해주세요."
        
        try:
            # 데이터프레임 컬럼 확인
            available_columns = list(self.df.columns)
            
            # 질의에 사용 가능한 컬럼이 포함되어 있는지 확인
            query_lower = query.lower()
            column_mentioned = any(col.lower() in query_lower for col in available_columns)
            
            if not column_mentioned:
                # 한국어 키워드 매핑
                korean_keywords = {
                    '생존': 'survived',
                    '나이': 'age', 
                    '성별': 'sex',
                    '등급': 'pclass',
                    '요금': 'fare',
                    '승선': 'embarked',
                    '형제자매': 'sibsp',
                    '부모자식': 'parch'
                }
                
                keyword_found = any(keyword in query for keyword in korean_keywords.keys())
                if not keyword_found:
                    return f"분석할 수 없습니다. 사용 가능한 컬럼: {', '.join(available_columns)}"
            
            # 에이전트에게 질의
            response = self.agent.run(query)
            return response
            
        except Exception as e:
            return f"분석 중 오류가 발생했습니다: {str(e)}"
    
    def create_visualization(self, viz_type, x_column, y_column=None):
        """시각화 생성"""
        try:
            plt.figure(figsize=(10, 6))
            
            if viz_type == "히스토그램":
                if x_column in self.df.columns:
                    plt.hist(self.df[x_column].dropna(), bins=30, alpha=0.7)
                    plt.title(f'{x_column} 분포')
                    plt.xlabel(x_column)
                    plt.ylabel('빈도')
                else:
                    plt.text(0.5, 0.5, f'컬럼 "{x_column}"을 찾을 수 없습니다.', 
                             horizontalalignment='center', verticalalignment='center')
                    plt.title('오류')
            
            elif viz_type == "박스플롯":
                if x_column in self.df.columns:
                    self.df.boxplot(column=x_column)
                    plt.title(f'{x_column} 박스플롯')
                else:
                    plt.text(0.5, 0.5, f'컬럼 "{x_column}"을 찾을 수 없습니다.', 
                             horizontalalignment='center', verticalalignment='center')
                    plt.title('오류')
            
            elif viz_type == "산점도":
                if x_column in self.df.columns and y_column in self.df.columns:
                    plt.scatter(self.df[x_column], self.df[y_column], alpha=0.6)
                    plt.xlabel(x_column)
                    plt.ylabel(y_column)
                    plt.title(f'{x_column} vs {y_column}')
                else:
                    plt.text(0.5, 0.5, f'컬럼을 확인해주세요.', 
                             horizontalalignment='center', verticalalignment='center')
                    plt.title('오류')
            
            elif viz_type == "막대그래프":
                if x_column in self.df.columns:
                    value_counts = self.df[x_column].value_counts()
                    plt.bar(range(len(value_counts)), value_counts.values)
                    plt.xticks(range(len(value_counts)), value_counts.index, rotation=45)
                    plt.title(f'{x_column} 개수')
                    plt.ylabel('개수')
                else:
                    plt.text(0.5, 0.5, f'컬럼 "{x_column}"을 찾을 수 없습니다.', 
                             horizontalalignment='center', verticalalignment='center')
                    plt.title('오류')
            
            elif viz_type == "생존율 분석":
                if 'survived' in self.df.columns and x_column in self.df.columns:
                    survival_rate = self.df.groupby(x_column)['survived'].mean()
                    plt.bar(range(len(survival_rate)), survival_rate.values)
                    plt.xticks(range(len(survival_rate)), survival_rate.index, rotation=45)
                    plt.title(f'{x_column}별 생존율')
                    plt.ylabel('생존율')
                    plt.ylim(0, 1)
                else:
                    plt.text(0.5, 0.5, '생존율 분석을 위해 survived 컬럼과 분석 컬럼이 필요합니다.', 
                             horizontalalignment='center', verticalalignment='center')
                    plt.title('오류')
            
            plt.tight_layout()
            
            # 이미지를 메모리에 저장
            buf = io.BytesIO()
            plt.savefig(buf, format='png', dpi=300, bbox_inches='tight')
            buf.seek(0)
            
            # PIL Image로 변환
            img = Image.open(buf)
            
            plt.close()
            
            return img
            
        except Exception as e:
            plt.figure(figsize=(8, 6))
            plt.text(0.5, 0.5, f'시각화 생성 중 오류: {str(e)}', 
                     horizontalalignment='center', verticalalignment='center')
            plt.title('오류')
            
            buf = io.BytesIO()
            plt.savefig(buf, format='png')
            buf.seek(0)
            img = Image.open(buf)
            plt.close()
            
            return img
    
    def get_data_info(self):
        """데이터 정보 반환"""
        if self.df is None:
            return "데이터가 로드되지 않았습니다."
        
        info = f"""
        **데이터셋 정보**
        - 총 행 수: {len(self.df)}
        - 총 열 수: {len(self.df.columns)}
        - 컬럼: {', '.join(self.df.columns)}
        
        **결측치 정보**
        {self.df.isnull().sum().to_string()}
        
        **기본 통계**
        {self.df.describe().to_string()}
        """
        
        return info

# TitanicAnalyzer 인스턴스 생성
analyzer = TitanicAnalyzer(df)

def analyze_text_query(query):
    """텍스트 질의 분석 함수"""
    return analyzer.analyze_query(query)

def create_chart(viz_type, x_col, y_col):
    """차트 생성 함수"""
    return analyzer.create_visualization(viz_type, x_col, y_col)

def get_dataset_info():
    """데이터셋 정보 반환"""
    return analyzer.get_data_info()

# Gradio 인터페이스 생성
def create_gradio_interface():
    with gr.Blocks(title="타이타닉 데이터 분석기", theme=gr.themes.Soft()) as demo:
        gr.Markdown("# 🚢 타이타닉 데이터 분석기")
        gr.Markdown("AI 에이전트를 활용한 타이타닉 데이터 분석 및 시각화 도구")
        
        with gr.Tabs():
            # 텍스트 질의 탭
            with gr.TabItem("💬 AI 질의 응답"):
                gr.Markdown("### 타이타닉 데이터에 대한 질문을 자연어로 입력하세요")
                
                with gr.Row():
                    with gr.Column(scale=3):
                        query_input = gr.Textbox(
                            label="질문 입력",
                            placeholder="예: 생존율이 가장 높은 등급은 무엇인가요?",
                            lines=3
                        )
                        query_btn = gr.Button("분석하기", variant="primary")
                    
                    with gr.Column(scale=2):
                        gr.Markdown("""
                        **예시 질문들:**
                        - 전체 생존율은 얼마인가요?
                        - 성별에 따른 생존율 차이는?
                        - 등급별 평균 요금은?
                        - 나이대별 생존 현황은?
                        """)
                
                analysis_output = gr.Textbox(
                    label="분석 결과",
                    lines=10,
                    max_lines=20
                )
                
                query_btn.click(
                    fn=analyze_text_query,
                    inputs=query_input,
                    outputs=analysis_output
                )
            
            # 시각화 탭
            with gr.TabItem("📊 시각화"):
                gr.Markdown("### 타이타닉 데이터 시각화")
                
                with gr.Row():
                    with gr.Column():
                        viz_type = gr.Dropdown(
                            choices=["히스토그램", "박스플롯", "산점도", "막대그래프", "생존율 분석"],
                            label="차트 유형",
                            value="히스토그램"
                        )
                        
                        x_column = gr.Dropdown(
                            choices=list(analyzer.df.columns) if analyzer.df is not None else [],
                            label="X축 컬럼",
                            value=list(analyzer.df.columns)[0] if analyzer.df is not None else None
                        )
                        
                        y_column = gr.Dropdown(
                            choices=list(analyzer.df.columns) if analyzer.df is not None else [],
                            label="Y축 컬럼 (산점도용)",
                            value=list(analyzer.df.columns)[1] if analyzer.df is not None and len(analyzer.df.columns) > 1 else None
                        )
                        
                        viz_btn = gr.Button("차트 생성", variant="primary")
                    
                    with gr.Column():
                        chart_output = gr.Image(
                            label="생성된 차트",
                            type="pil"
                        )
                
                viz_btn.click(
                    fn=create_chart,
                    inputs=[viz_type, x_column, y_column],
                    outputs=chart_output
                )
            
            # 데이터 정보 탭
            with gr.TabItem("📋 데이터 정보"):
                gr.Markdown("### 데이터셋 기본 정보")
                
                info_btn = gr.Button("정보 새로고침", variant="secondary")
                data_info = gr.Textbox(
                    label="데이터셋 정보",
                    value=get_dataset_info(),
                    lines=20,
                    max_lines=30
                )
                
                info_btn.click(
                    fn=get_dataset_info,
                    outputs=data_info
                )
        
        gr.Markdown("""
        ### 사용법 안내
        1. **AI 질의 응답**: 자연어로 데이터에 대한 질문을 입력하면 AI가 분석해서 답변합니다.
        2. **시각화**: 드롭다운에서 차트 유형과 컬럼을 선택하여 시각화를 생성합니다.
        3. **데이터 정보**: 현재 로드된 데이터셋의 기본 정보를 확인할 수 있습니다.
        
        ⚠️ **주의사항**: AI 질의 응답 기능을 사용하려면 OpenAI API 키가 필요합니다.
        """)
    
    return demo

demo = create_gradio_interface()
demo.launch(
    share=True,
    debug=True
)

/Users/kwangsiklee/miniforge3/envs/langchain/lib/python3.10/site-packages/langchain_experimental/agents/agent_toolkits/pandas/base.py:283: UserWarning: Received additional kwargs {'handle_parsing_errors': True} which are no longer supported.
  warnings.warn(


Agent 설정 완료
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://14e9d9a6abd3c8d175.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)




> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "male_survival_rate = df[df['성별'] == 'male']['생존여부'].mean()\nfemale_survival_rate = df[df['성별'] == 'female']['생존여부'].mean()\nmale_survival_rate, female_survival_rate"}`


(np.float64(0.18890814558058924), np.float64(0.7420382165605095))남자의 생존율은 약 18.89%이고, 여자의 생존율은 약 74.20%입니다.

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "import pandas as pd\nimport matplotlib.pyplot as plt\nimport seaborn as sns\n\n# 나이대별로 그룹화하여 생존현황을 집계\nage_bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]\nage_labels = ['0-10', '11-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100']\ndf['나이대'] = pd.cut(df['나이'], bins=age_bins, labels=age_labels, right=False)\n\n# 나이대별 생존현황 집계\nsurvival_by_age = df.groupby(['나이대', '생존여부']).size().unstack(fill_value=0)\n\n# 시각화\nplt.figure(figsize=(12, 6))\nsns.barplot(data=survival_by_age.reset_index(), x='나이대', 

<string>:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
<string>:1: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown


나이대별 생존현황을 시각화한 결과입니다. 각 나이대에 따라 생존한 승객과 생존하지 못한 승객의 수를 비교할 수 있습니다. 생존한 승객은 연한 파란색으로, 생존하지 못한 승객은 연한 분홍색으로 표시되었습니다.

> Finished chain.
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://14e9d9a6abd3c8d175.gradio.live
